This ipynb is for fine tuning and testing GPT-3 model using OpenAI API and sentiment140 dataset. The API key is omitted on this repo for security reasons, so the code won't be able to run without it.

In [ ]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 16.8 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=ab166ead3b915ca80681d961c221e414c03f6d68de9addee601e391e8f549781
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai


In [ ]:
#@title OpenAI API Key
import os

os.environ["OPENAI_API_KEY"] = ''

In [ ]:
#@title Imported Packages and Libraries

# API
import requests
import openai
import os

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Word2vec
import gensim

# Keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
SEQUENCE_LENGTH = 300
TRAIN_SIZE = 0.8

In [ ]:
# UPLOAD THE FOLLOWING FILE: training.1600000.processed.noemoticon.csv

from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = "latin-1", low_memory=False)
  
df_train, df_test = train_test_split(df, test_size=1-TRAIN_SIZE, random_state=42)

df_train = df_train[['text','target']]
df_test = df_test[['text','target']]

# Reassigning 4='positive' 0='negative'
df_train['target'] = df_train['target'].replace({4:'positive',0:'negative'})
df_test['target'] = df_test['target'].replace({4:'positive',0:'negative'})

In [ ]:
# Data Formatting for OpenAPI endpoint (see https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for details)
df_train = df_train.rename(columns={'text':'prompt','target':'completion'})
df_test = df_test.rename(columns={'text':'prompt','target':'completion'})

train_prompts = [p+'\n\n###\n\n' for p in df_train['prompt']]
test_prompts = [p+'\n\n###\n\n' for p in df_test['prompt']]
df_train['prompt'] = train_prompts
df_test['prompt'] = test_prompts

train_comp = [' '+c+'###' for c in df_train['completion']]
test_comp = [' '+c+'###' for c in df_test['completion']]
df_train['completion'] = train_comp
df_test['completion'] = test_comp

In [ ]:
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

TRAIN size: 40000
TEST size: 10000


In [ ]:
df_train.head()

,prompt,completion
39087,@MsBrownSknPopn Not at all. I'm The Man. \n\n#...,positive###
30893,@CindiCupcaking headin' east - Boston \n\n###\n\n,positive###
45278,is soon off to niagara-on-the-lake trip with c...,positive###
16398,@skinnymarie and guess what ! ive got 20 % ! w...,negative###
13653,needs Mom probs to ease up really soon... htt...,negative###


In [ ]:
df_test.head()

,prompt,completion
33553,...getting out of Hamburg...just leaving hotel...,positive###
9427,@kerri_louise aww i like the rain but the sun...,negative###
199,Depressed to learn the BNP won a EU Parliament...,negative###
12447,Try sleep again.. Really miss my beboonggg! h...,negative###
39489,@Fanny57 Then Dave jumped on me... wasn't dis...,positive###


In [ ]:
df_train.to_csv('train.csv', encoding='utf-8', index=False)
df_test.to_csv('test.csv', encoding='utf-8', index=False)

In [ ]:
!openai tools fine_tunes.prepare_data -f train.csv

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 40000 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 65 duplicated prompt-completion sets. These are rows: [3470, 4526, 5313, 6658, 9373, 11068, 11899, 12399, 12739, 12829, 14555, 14732, 16009, 16193, 16486, 17420, 19049, 19166, 19571, 20208, 20978, 21544, 22055, 22104, 22756, 22864, 23645, 23877, 24193, 24416, 24550, 24616, 24884, 25027, 26532, 26647, 26738, 26984, 27760, 28751, 29017, 29389, 30193, 30531, 31973, 32890, 32959, 33325, 33554, 33610, 33692, 33818, 34085, 34088, 35346, 35407, 35822, 35840, 36561, 36909, 37140, 37467, 38412, 38740, 39099]
- All prompts end with suffix `\n

In [ ]:
r1 = openai.File.create(file=open("train_prepared_train.jsonl"), purpose='fine-tune')
r2 = openai.File.create(file=open("train_prepared_valid.jsonl"), purpose='fine-tune')

print(r1)
print(r2)

{
  "bytes": 4963103,
  "created_at": 1670278028,
  "filename": "file",
  "id": "file-OamfjrrbClxppaTNWjcEO382",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}
{
  "bytes": 128253,
  "created_at": 1670278053,
  "filename": "file",
  "id": "file-p0fEHzhiGsbLIlWE9qIrfaOg",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


In [ ]:
#@title Create Fine Tune Job for GPT-3
training_data = r1['id']
validation_data= r2['id']

def create_model(suffix, base_model, training_data, validation_data,epochs):
  response = openai.FineTune.create(
      training_file=training_data,
      validation_file=validation_data,
      model=base_model,
      compute_classification_metrics=True,
      classification_n_classes=2,
      n_epochs=epochs,
      suffix=suffix,
      classification_positive_class=' positive###',
      )
  return response.to_dict()


In [ ]:
#@title CREATE FINE TUNE JOB
ft_response = create_model('Kenny','curie',training_data,validation_data,epochs=4)
print(ft_response)

{'object': 'fine-tune', 'id': 'ft-whtuzffJz8EwrrVKoJttjMYx', 'hyperparams': <OpenAIObject at 0x7f8f8623c8b0> JSON: {
  "batch_size": null,
  "classification_n_classes": 2,
  "classification_positive_class": " positive###",
  "compute_classification_metrics": true,
  "learning_rate_multiplier": null,
  "n_epochs": 4,
  "prompt_loss_weight": 0.01
}, 'organization_id': 'org-rvWBJysoXs405lvjgSp1U88F', 'model': 'curie', 'training_files': [<File file id=file-OamfjrrbClxppaTNWjcEO382 at 0x7f8f8623cc20> JSON: {
  "bytes": 4963103,
  "created_at": 1670278028,
  "filename": "file",
  "id": "file-OamfjrrbClxppaTNWjcEO382",
  "object": "file",
  "purpose": "fine-tune",
  "status": "processed",
  "status_details": null
}], 'validation_files': [<File file id=file-p0fEHzhiGsbLIlWE9qIrfaOg at 0x7f8f86244680> JSON: {
  "bytes": 128253,
  "created_at": 1670278053,
  "filename": "file",
  "id": "file-p0fEHzhiGsbLIlWE9qIrfaOg",
  "object": "file",
  "purpose": "fine-tune",
  "status": "processed",
  "stat

In [ ]:
job_id = ft_response['id']
print(job_id)

ft-whtuzffJz8EwrrVKoJttjMYx


In [ ]:
# Retrieve status of fine-tune job
def retrieve(job_id):
  return openai.FineTune.retrieve(id=job_id)

def cancel(job_id):
  return openai.FineTune.cancel(id=job_id)

In [ ]:
#@title CANCEL FINE-TUNE JOB

cancel(job_id)

InvalidRequestError: ignored

In [ ]:
retrieval = retrieve(job_id)
print(retrieval)

{
  "created_at": 1670278358,
  "events": [
    {
      "created_at": 1670278358,
      "level": "info",
      "message": "Created fine-tune: ft-whtuzffJz8EwrrVKoJttjMYx",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1670278360,
      "level": "info",
      "message": "Fine-tune costs $12.82",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1670278361,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1670278362,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1670279080,
      "level": "info",
      "message": "Completed epoch 1/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1670279715,
      "level": "info",
      "message": "Completed epoch 2/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1670280348,
      "level": "info",

In [ ]:
# get model results file
def results(result_file):
  byte = openai.File.download(id=result_file)
  csv = str(byte)[2:-1]
  csv = csv.replace('\\t', ',').replace('\\n', '\n')
  print(csv, file=open('results.csv', 'w'))

In [ ]:
result_id = retrieval['result_files'][0]['id']
model_id = retrieval['fine_tuned_model']
#results(result_id)
print(model_id)

curie:ft-personal:kenny-2022-12-05-22-56-57


In [ ]:
# Use this to get completion of given prompt and model
def completion(prompt,model):
  response = openai.Completion.create(
              model=model,
              prompt=prompt+"\n\n###\n\n",
              max_tokens=1,
              temperature=0
            )
  return response.to_dict()['choices'][0]['text'][1:]

In [ ]:
from openai.error import RateLimitError
import time
from tqdm import tqdm 

def test_accuracy(model):
  correct = 0
  total = 0
  
  for row in tqdm(df_test.itertuples(index=False)):
    total += 1
    # Can only perform 60 completions per minute
    while True:
      try:
        result = completion(row[0],model)
      except RateLimitError:
        print('Encountered RateLimitError: Waiting for 65 seconds...')
        print(f'Current accuracy:{correct/total}')
        time.sleep(65)
        continue
      break 

    c1 = 'positive' in result
    c2 = 'negative' in result 
    if not c1 ^ c2:
      raise Exception('Model did not generate valid completion')
      break
    elif result in row[1]:
      correct += 1
  return correct/len(df_test)


In [ ]:
test_accuracy(model_id)

43it [00:04, 14.62it/s]

Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8


85it [01:13, 12.36it/s]

Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8735632183908046


132it [02:22, 15.07it/s]

...

Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8885615603413247


9844it [4:13:29, 10.10s/it]

Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8885728796343322
Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8885728796343322
Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8885728796343322
Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8885728796343322
Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8885728796343322
Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8885728796343322
Encountered RateLimitError: Waiting for 65 seconds...
Current accuracy:0.8885728796343322


9844it [4:20:37,  1.59s/it]


KeyboardInterrupt: ignored